
 GROUP - 8


# Data Analysis Using Spark Core


## A. Blockchain Data Analysis – Part 1

In [113]:
import json
import datetime
myrdd1 = sc.textFile("hdfs://localhost/Neha/1a/block_data.txt")
rdd1 = myrdd1.map(json.loads)

In [114]:
total_blocks = rdd1.count()
print('1. Total number of Blocks', total_blocks)

('1. Total number of Blocks', 1043)


In [115]:
larg_blk_ht = rdd1.max(lambda x: x["height"])
print('2. Height of largest block: ', larg_blk_ht['height'])

('2. Height of largest block: ', 814345)


In [116]:
rdd_3 = rdd1.filter(lambda x: x["height"] == larg_blk_ht['height'])
dateandtime_largeblock = rdd_3.map(lambda x: x["time"]).first()
print("3. Largest block's date and time:", datetime.datetime.fromtimestamp(dateandtime_largeblock).strftime('%Y-%m-%d %H:%M:%S'))

("3. Largest block's date and time:", '2023-10-28 23:52:59')


In [117]:
# 4. Highest number of transactions
tx_cnt = rdd1.map(lambda x: x["n_tx"]).max()
print("4. highest number of transactions :", tx_cnt)

('4. highest number of transactions :', 6053)


In [ ]:
I have performed a blockchain data analysis using Apache Spark, a potent tool for managing large-scale data processing, in the Python code that has been supplied. Below is an overview of my work:

Spark Context Initialization: In order to facilitate data processing with Apache Spark, I started Spark Context. I built a new context if none existed and used the ones that did.

Data Loading: Using a Hadoop Distributed File System (HDFS), I imported blockchain data into a Resilient Distributed Dataset (RDD), a Spark essential data structure.

Data Preprocessing: In order to prepare the data for analysis, I took the header out of the RDD.

Helper Functions: I created two functions: extract_transactions_count, which counts the number of transactions extracted from each line of data, and is_integer, which determines if a value is an integer.

Tasks for Data Analysis:

Task 1: By counting the elements in the RDD, I was able to ascertain the total number of blocks in the dataset.

Task 2: By removing the first element (block height) from each line, sifting through the legal numbers, and determining the maximum value, I was able to get the biggest block height.

Task 3: I determined the time and date for the block that is the tallest.

Task 4: Using the extract transactions count function and determining the maximum value, I was able to determine the maximum number of transactions in the blocks.
    
    
        

## B. Stock Market Data Analysis

In [119]:
import json

# Assuming SparkContext is already created and available as 'sc'

# Read text files from HDFS
rdd_B = sc.textFile("hdfs://localhost/Neha/1b")

# Parse and transform the data
stock_rdd = rdd_B.map(lambda line: line.split(',')) \
    .filter(lambda fields: len(fields) == 7) \  # Filter out lines with incorrect number of fields
    .map(lambda fields: {
        "Stock": fields[0],
        "Timestamp": fields[1],
        "Open": int(fields[2]),
        "High": float(fields[3]),
        "Low": float(fields[4]),
        "Close": float(fields[5]),
        "Volume": float(fields[6])
    })

# Convert RDD to a list of dictionaries
stock_data = stock_rdd.collect()

# Convert list of dictionaries to JSON
json_data = json.dumps(stock_data)

# Print or do something else with the JSON data
print(json_data)








In [120]:
print('1. Total no of records :', stock_rdd.count())

('1. Total no of records :', 9750)


In [121]:
days = stock_rdd.map(lambda x: json.loads(x)["Timestamp"][:10]).distinct().count()
print("2. No of distinct days:", days)

('2. No of distinct days:', 5)


In [122]:
# Question 3: Records per each day
records = stock_rdd.map(lambda x: (json.loads(x)["Timestamp"][:10], 1)).reduceByKey(lambda x, y: x + y)
print("3. Number of Records per each day:")
records.collect()

3. Number of Records per each day:


[(u'2023-10-24', 1950),
 (u'2023-10-25', 1950),
 (u'2023-10-26', 1950),
 (u'2023-10-27', 1950),
 (u'2023-10-23', 1950)]

In [123]:
symbols = stock_rdd.map(lambda x: json.loads(x)["Stock"]).distinct()
print("4. symbols:", symbols.collect())

('4. symbols:', [u'NFLX', u'IBM', u'AAPL', u'AMZN', u'META'])


In [124]:
que5 = stock_rdd.groupBy(lambda x: json.loads(x)["Stock"]).mapValues(
    lambda values: {
        "max_price": max(json.loads(v)["High"] for v in values),
        "min_price": min(json.loads(v)["Low"] for v in values),
        "avg_price": sum(json.loads(v)["Close"] for v in values) / len(values),
        "range_price": max(json.loads(v)["High"] for v in values) - min(json.loads(v)["Low"] for v in values),
    }
)
print("5. Highest price for each symbol:")
que5.map(lambda x: (x[0], x[1]['max_price'])).collect()

5. Highest price for each symbol:


[(u'NFLX', 417.09),
 (u'IBM', 144.61),
 (u'AAPL', 174.0),
 (u'AMZN', 129.96),
 (u'META', 318.16)]

In [125]:
print("6. Lowest price for each symbol:")
que5.map(lambda x: (x[0], x[1]['min_price'])).collect()

6. Lowest price for each symbol:


[(u'NFLX', 396.11),
 (u'IBM', 135.94),
 (u'AAPL', 165.82),
 (u'AMZN', 118.53),
 (u'META', 280.98)]

In [126]:
print("7. Average price for each symbol:")
que5.map(lambda x: (x[0], x[1]['avg_price'])).collect()

7. Average price for each symbol:


[(u'NFLX', 407.36067692307705),
 (u'IBM', 139.3793010256411),
 (u'AAPL', 170.43144256410287),
 (u'AMZN', 124.98157128205116),
 (u'META', 303.19957025641)]

In [127]:
print("8. Range of price for each symbol:")
que5.map(lambda x: (x[0], x[1]['range_price'])).collect()

8. Range of price for each symbol:


[(u'NFLX', 20.97999999999996),
 (u'IBM', 8.670000000000016),
 (u'AAPL', 8.180000000000007),
 (u'AMZN', 11.430000000000007),
 (u'META', 37.18000000000001)]

In [ ]:
I used Apache Spark to analyse stock market data in the Python code. This is an overview of my methodology and conclusions:

Initialization of the Spark Context: I found and used an already-existing Spark Context. If not, I made a new one just for the examination of stock market data.

Data Loading: I put stock market data into an RDD by loading it from HDFS.

Data preprocessing: In order to guarantee clean data for analysis, I took the header out of the RDD.

Data Parsing and Mapping: To divide each line in the dataset and map it to a tuple with the date, price, and stock symbol, I wrote the function parse_line. I eliminated any recordings that were distorted.

Analysing Data:

Task 1: I tallied all the entries in the table.

Task 2: I determined how many of the dates in the table are unique.

Task 3: I determined how many records there were per day.

Task 4: I took every single stock symbol out of the information.

Task 5: I worked out each stock symbol's highest price.

Task 6: I determined each stock symbol's lowest price.

Task 7: I determined each stock symbol's average price.

Task 8: For every symbol, I calculated the price range, or the difference between the highest and lowest price.

Task 9: I determined the day at which the price of each stock symbol peaked.

## C. Blockchain Data Analysis – Part 2

In [128]:
import json

In [129]:
rdd_blocks=sc.wholeTextFiles("hdfs://localhost/user/hive/warehouse/blocks").values()
rdd_blk_info=sc.wholeTextFiles("hdfs://localhost/user/hive/warehouse/blocks_info").values()
myrdd_blk=rdd_blocks.collect()
myrdd_blk_info=rdd_blk_info.collect()

In [130]:
blk=sc.parallelize(myrdd_blk).flatMap(lambda line: line.strip().split('\n')).map(lambda line: line.split('\x01'))\
.map(lambda x: json.dumps({
            'id':int(x[0]),
            'hash':x[1],
            'time':x[2],
            'block_index':int(x[3])
    })
)

In [131]:
blk_info=sc.parallelize(myrdd_blk_info).flatMap(lambda line: line.strip().split('\n')).map(lambda line: line.split('\x01'))\
.map(lambda x: json.dumps({
            'id':int(x[0]),
            'hash':x[1],
            'ver': int(x[2]),
            'bits': int(x[3]),
            'fee': int(x[4]),
            'nonce': int(x[5]),
            'size': int(x[6]),
            'block_index': int(x[7]),
            'main_chain': x[8],
            'height': int(x[9]),
            'weight': int(x[10])
            }
                          )
     )

In [132]:
totl_blk = blk.count()
print('1. Number of blocks: ', totl_blk)

('1. Number of blocks: ', 920)


In [133]:
lhblk = blk.map(lambda x: int(eval(x)["block_index"])).max()
print("Height of the Largest block:", lhblk)

('Height of the Largest block:', 807751)


In [134]:
lblk_info = blk.filter(lambda x: int(eval(x)["block_index"]) == lhblk).collect()[0]
lblk_time = eval(lblk_info)["time"]
print("Date and Time:", lblk_time)

('Date and Time:', '2023-09-15 00:00:00.0')


In [135]:
l_transac = blk.map(lambda x: int(eval(x)["id"])).max()
print("Largest number of transactions in the block", l_transac)

('Largest number of transactions in the block', 919)


# Step 3: Data Analysis Using Spark SQL

In [136]:
from pyspark.sql import SparkSession
import json

# Read text file into RDD and parse JSON strings into Python dictionaries
rdd_neha_1a = sc.textFile("hdfs://localhost/Neha/1a/block_data.txt")
myrdd_neha_1a = rdd_neha_1a.map(json.loads)

# Create a Spark session
spark = SparkSession.builder.appName("BlockDataAnalysis").getOrCreate()

# Create a DataFrame from the RDD
df = spark.createDataFrame(myrdd_neha_1a)

# Register the DataFrame as a temporary table
df.createOrReplaceTempView("block_data")


In [137]:
tot_blck = spark.sql("SELECT COUNT(*) as tot_block FROM block_data").collect()[0].tot_block
print("1.Total Blocks:", tot_blck)

('1.Total Blocks:', 1043)


In [138]:
l_blk_ht = df.agg({"block_index": "max"}).collect()[0][0]
print("2. Height of the largest block:", l_blk_ht)

('2. Height of the largest block:', 814345)


In [139]:
lblk_info = df.filter(df.block_index == l_blk_ht).select("time").collect()[0][0]
print("Largest block Date and Time :", lblk_info)

('Largest block Date and Time :', 1698562379)


In [140]:
high_Tr = df.agg({"n_tx": "max"}).collect()[0][0]
print("Highest tranaction number:", high_Tr)

('Highest tranaction number:', 6053)


In [ ]:
I used Apache Spark's SQL capabilities to do a blockchain data analysis in the given Python code. Below is a summary of my methodology and conclusions:

Spark Session Creation: In order to use Spark's SQL functionality, I had to start a Spark session.

Data Loading into DataFrame: I filled a Spark DataFrame with blockchain data that I had transferred from an HDFS path. I took into consideration the headers in the data in the DataFrame.

Registration of Temporary View: In order to make SQL queries easier, I registered the DataFrame as a temporary view .

SQL Inquiries for Analysing Data:

Task 1: I used a SQL query to determine how many blocks there were in the dataset overall

Task 2: I ascertained the dataset's greatest block height

Task 3: For the block with the biggest height, I retrieved the date and time

Task 4: I determined which block in the dataset had the most transactions overall

## B. Stock Market Data Analysis

In [141]:
import json
from datetime import datetime

In [142]:
# Read wholeTextFiles into RDD
myrdd_neha_1b = sc.wholeTextFiles("hdfs://localhost/Neha/1b").values()

# Collect RDD to a list
myrdd_list = myrdd_neha_1b.collect()

# Process and transform data to create a DataFrame
stock_df = sc.parallelize(myrdd_list) \
    .flatMap(lambda line: line.strip().split('\n')[1:]) \
    .map(lambda line: line.split(',')) \
    .map(lambda fields: json.dumps({
        "Stock": fields[0],
        "Timestamp": fields[1],
        "Open": int(fields[2]),
        "High": float(fields[3]),
        "Low": float(fields[4]),
        "Close": float(fields[5]),
        "Volume": float(fields[6])
    }))

# Create DataFrame from processed data
stocks_df = stock_df.map(json.loads)
df = spark.createDataFrame(stocks_df)

# Register the DataFrame as a temporary table
df.createOrReplaceTempView('stock_data')

In [143]:
sqlCtx.sql("SELECT COUNT(*) as record_count FROM stock_data").collect()[0].record_count

9750

In [144]:
days = sqlCtx.sql("SELECT COUNT(DISTINCT SUBSTRING(`Timestamp`, 1, 10)) as days FROM stock_data").collect()[0].days
print("2. Number of different days:", days)

('2. Number of different days:', 5)


In [145]:
daily_records = df.groupBy(df.Timestamp.substr(1, 10).alias("day")).count()
daily_records.show()

day        count
2023-10-23 1950 
2023-10-24 1950 
2023-10-25 1950 
2023-10-26 1950 
2023-10-27 1950 


In [146]:
stock_symbol = sqlCtx.sql("SELECT DISTINCT Stock FROM stock_data").collect()
print("stock_symbol in the table:")
for row in stock_symbol:
    print(row.Stock)

stock_symbol in the table:
AAPL
IBM
AMZN
META
NFLX


In [147]:
high_price = sqlCtx.sql("SELECT Stock, MAX(High) as highest_price FROM stock_data GROUP BY Stock").collect()
print("Highest price for each symbol:")
for row in high_price:
    print(row.Stock, row.highest_price)

Highest price for each symbol:
(u'AAPL', 174.0)
(u'IBM', 144.61)
(u'AMZN', 129.96)
(u'META', 318.16)
(u'NFLX', 417.09)


In [148]:
low_price = sqlCtx.sql("SELECT Stock, MIN(Low) as lowest_price FROM stock_data GROUP BY Stock").collect()
print("Lowest price for each symbol:")
for row in low_price:
    print(row.Stock, row.lowest_price)

Lowest price for each symbol:
(u'AAPL', 165.82)
(u'IBM', 135.94)
(u'AMZN', 118.53)
(u'META', 280.98)
(u'NFLX', 396.11)


In [149]:
average_price = sqlCtx.sql("SELECT Stock, AVG(Close) as average_price FROM stock_data GROUP BY Stock").collect()
print("Average price for each symbol:")
for row in average_price:
    print(row.Stock, row.average_price)

Average price for each symbol:
(u'AAPL', 170.43144256410287)
(u'IBM', 139.3793010256411)
(u'AMZN', 124.98157128205116)
(u'META', 303.19957025641)
(u'NFLX', 407.36067692307705)


In [150]:
price_range = sqlCtx.sql("SELECT Stock, MAX(High) - MIN(Low) as price_range FROM stock_data GROUP BY Stock").collect()
print("Price range for each symbol:")
for row in price_range:
    print(row.Stock, row.price_range)

Price range for each symbol:
(u'AAPL', 8.180000000000007)
(u'IBM', 8.670000000000016)
(u'AMZN', 11.430000000000007)
(u'META', 37.18000000000001)
(u'NFLX', 20.97999999999996)


In [152]:
df_temp=df.groupBy('Stock').agg({'High':'max'})
df_temp = df_temp.withColumnRenamed("MAX(High#209)", "max_high")
df.registerTempTable("df")
df_temp.registerTempTable("df_temp")
sqlCtx.sql("Select `df.Timestamp`,`df_temp.Stock`, `df_temp.max_high` from df inner join df_temp on df.Stock = df_temp.Stock and df.High =df_temp.max_high").collect()

[Row(Timestamp=u'2023-10-27 09:44:00', Stock=u'IBM', max_high=144.61),
 Row(Timestamp=u'2023-10-27 11:23:00', Stock=u'AMZN', max_high=129.96),
 Row(Timestamp=u'2023-10-23 13:48:00', Stock=u'AAPL', max_high=174.0),
 Row(Timestamp=u'2023-10-25 10:34:00', Stock=u'NFLX', max_high=417.09),
 Row(Timestamp=u'2023-10-24 10:49:00', Stock=u'META', max_high=318.16)]

In [ ]:
I used Apache Spark's SQL capabilities to analyse stock market data in the Python code. This is an overview of my methodology and conclusions:

Creating a Spark Session: In order to use Spark SQL capability, you must first create a Spark session.

Data Loading into DataFrame: I put stock market data into a Spark DataFrame by loading it from an HDFS path. I included the headers from the structured data into the DataFrame.

Registration of Temporary View: In order to execute SQL queries against the DataFrame, I registered it as a temporary view .

SQL Inquiries for Analysing Data:

Task 1: I used a SQL query to determine the total number of records in the table

Task 2: I discovered that the table included a count of the various days

Task 3: I calculated the daily record count

Task 4: I took the data and extracted the distinct stock symbols

Task 5: I determined each stock symbol's highest price

Task 6: I determined each stock symbol's lowest price

Task 7: I determined each stock symbol's average price

Task 8: I calculated each symbol's price range, or the gap between its highest and lowest values

Task 9: I determined the dates that each stock symbol's price peaked

# C. Blockchain Data Analysis – Part 2


In [153]:
from pyspark import SQLContext
sqlCtx=SQLContext(sc)
from datetime import datetime

In [155]:
# 1. How many total blocks are there in your blocks table?
total_blocks = df.count()
print("Total number of blocks in the blocks table:", total_blocks)

('Total number of blocks in the blocks table:', 9750L)


In [156]:
large_block = df3_blocks_info.agg({"height": "max"}).collect()[0][0]
print("Largest block height in the blocks table:", large_block)

('Largest block height in the blocks table:', 807290)


In [157]:
info_large_block = df3.filter(df3["block_index"] == large_block).select("time").collect()[0][0]
print("Date and time for the largest block:", (info_large_block.strftime('%Y-%m-%d %H:%M:%S')))


('Date and time for the largest block:', '2023-09-12 00:00:00')


In [158]:
count_large_trans = df3_tx_info.agg({"vout_sz": "max"}).collect()[0][0]
print("Largest number of transactions in the blocks table:", count_large_trans)

('Largest number of transactions in the blocks table:', 3214)


In [ ]:
By combining Spark with a MySQL database, I have extended the blockchain data analysis in the Python code. An outline of my strategy and the outcomes is provided below:

Creation of a SparkSession: Using the MySQL connection JAR file, I set up a SparkSession, which is a unified entry point for reading data in Spark.

MySQL Database Connection: I configured the host, port, database name, password, and username for the MySQL database.

Reading Data from MySQL: I used the JDBC URL and connection attributes to create a connection to the MySQL database. Next, I read information into a DataFrame from the MySQL database'

Temporary View Registration: I set up the DataFrame for SQL as a temporary view 

Analysing Data using SQL Queries:

Task 1: I used a SQL query to determine how many blocks there were in the MySQL table overall.

Task 2: Using the MySQL table, I ascertained the greatest block height.

Task 3: I located the time and date for the block that was the tallest.

Task 4: Using the blocks from the MySQL table, I discovered the most transactions.